In [74]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.tools.plotting import scatter_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [84]:
df = pd.read_json('data.json')

In [85]:
df.fillna(0, inplace=True)

In [86]:
df['Fraud'] = ((df['acct_type'] == 'fraudster_event') | (df['acct_type'] == 'fraudster') | \
               (df['acct_type'] == 'locked') | (df['acct_type'] == 'tos_lock') | \
               (df['acct_type'] == 'tos_warn') | (df['acct_type'] == 'fraudster_att'))

In [87]:
def balancer(df, label, margin = 0.4, size = 500):
    yes = df[df[label] == 1]
    no = df[df[label] == 0]
    return yes.sample(int(size * margin),axis =0).append(no.sample(size - int(size * margin),axis =0))

In [88]:
df = balancer(df, 'Fraud', margin = 0.4, size = 500)

In [89]:
df['ec'] = pd.to_datetime(df['event_created'],unit='s')
df['es'] = pd.to_datetime(df['event_start'],unit='s')
df['ep'] = pd.to_datetime(df['event_published'],unit='s')
df['ec'] = df['ec'].dt.date
df['es'] = df['es'].dt.date
df['ep'] = df['ep'].dt.date
df['all_equal'] = ((df['es']==df['ec']) & (df['es']==df['ep']))
df.drop(['es', 'ec','ep'], axis=1, inplace=True)

In [90]:
df['all_equal'].value_counts()

False    477
True      23
Name: all_equal, dtype: int64

In [92]:
df.T

,14297,4787,3259,12242,1633,7690,3198,9582,11803,11234,...,5654,8486,3923,7136,1637,2393,8031,8263,4225,13401
acct_type,fraudster_event,tos_lock,fraudster_event,fraudster_event,fraudster,fraudster,fraudster_event,fraudster,fraudster_event,fraudster_event,...,premium,premium,premium,premium,premium,premium,premium,premium,premium,premium
approx_payout_date,1362859200,1357430400,1353378600,1347922800,1333519200,1339934400,1344207600,1314838800,1356220800,1349046000,...,1340915400,1377923400,1372827600,1335729600,1338249600,1374861600,1310846400,1369776600,1351182600,1374557400
body_length,6297,3724,1788,0,0,790,68,261,2349,300,...,1228,569,225,1924,2958,471,130,0,3226,1272
channels,5,8,8,0,6,5,13,8,5,5,...,6,0,6,11,5,0,8,8,8,6
country,GB,US,A1,US,CA,US,GB,US,US,US,...,GB,,US,US,AU,US,US,US,US,GB
currency,GBP,USD,GBP,USD,CAD,USD,GBP,USD,USD,USD,...,GBP,USD,USD,USD,AUD,USD,USD,USD,USD,GBP
delivery_method,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
description,"<div style=""text-align: center;""><strong></str...","<center>\r\n<p><a href=""http://gametightnye.cr...","<p style=""margin-top: 15px; margin-bottom: 15p...",,,"<p><span class=""vevent""><span class=""descripti...",<p>Its going to be an atmosphere where you can...,<p>4 Rounds of Game/30 minutes<br />EF: $85 ($...,"<p class=""rich-normal"" style=""line-height: nor...","<p>In Forex, if you gamble like in a casino, y...",...,"<p class=""MsoBodyText"" style=""text-align: left...",<p>Fire ceremonies are a wonder-filled way to ...,"<p><span style=""font-size: medium;"">View more ...","<p><span style=""font-size: medium;""><font face...",<p><strong>Statistics &amp; Data Analysis Fund...,"<p style=""text-align: left;""><span style=""colo...","<p>This workshop will cover basic editing, sho...",,"<p><span style=""font-family: Times New Roman; ...","<p style=""margin: 0px 5px; padding: 0px; font-..."
email_domain,thinktankconsultancy.com,hotmail.com,consultant.com,aol.com,yahoo.com,yopmail.com,yahoo.com,ymail.com,hotmail.com,lmtexformula.com,...,gmail.com,sbcglobal.net,cityswarmchicago.com,retailmaine.org,gmail.com,media.mit.edu,yahoo.com,internbridge.com,westefc.org,gmail.com
event_created,1357325405,1356511378,1352727236,1347294973,1333030964,1339484284,1343141285,1314255958,1355554930,1348296276,...,1333538541,1376877298,1369856234,1333391430,1334294441,1367345994,1299957590,1351801357,1348164628,1368727066


In [61]:
df = df[np.isfinite(df['org_facebook'])]
df = df[np.isfinite(df['event_published'])]
df = df[np.isfinite(df['sale_duration'])]
df = df.dropna(subset=['country']) 
df = df.dropna(subset=['delivery_method']) 
df = df[np.isfinite(df['venue_longitude'])]
df = df[np.isfinite(df['sale_duration'])]
df = df.dropna(subset=['country']) 
df = df.dropna(subset=['delivery_method']) 

In [63]:
keep_col = ['approx_payout_date', 'body_length', 'channels', 'country', 'currency', \
            'delivery_method', 'event_created', 'event_end', 'event_published', 'event_start', \
            'fb_published', 'gts', 'has_analytics', 'has_logo', 'listed', \
            'name_length', 'num_order', 'num_payouts', 'org_facebook', 'org_twitter', \
            'payout_type', 'sale_duration', 'sale_duration2', 'show_map', 'user_age', \
            'user_created', 'user_type', 'venue_country', 'venue_latitude', 'venue_longitude', \
            'venue_state', 'Fraud', 'all_equal']

In [65]:
df1 = df[keep_col]
df.drop

<bound method DataFrame.drop of              acct_type  approx_payout_date  body_length  channels country  \
815           tos_warn          1361444400         2619         6      US   
7475   fraudster_event          1379307600            0         0      US   
12780  fraudster_event          1356523200          989         0      GB   
9864   fraudster_event          1364619600           76         0      GB   
8515         fraudster          1301551200         4076         0      GB   
737    fraudster_event          1351252800           47         0      US   
6637         fraudster          1333605600          108         6      US   
12863  fraudster_event          1367560800            0         8           
9644            locked          1331274600         1939         0      US   
2670   fraudster_event          1338634800          990         5      US   
4140   fraudster_event          1334206800         2764         6      US   
599          fraudster          1324681200  

In [66]:
dummy_col = ['country', 'currency', 'listed', 'payout_type', 'venue_country', 'venue_state']

In [67]:
for col in dummy_col:
    df1 = pd.concat([df1, pd.get_dummies(df[col])], axis=1)
    del df1[col]
    
df1.shape

(500, 178)

In [71]:
df1['all_equal'].unique()

array([False], dtype=object)